## Basic RL usage

### Initializing environments


#### **Environment settings**

- Initializing an environment is done with the `BaseEnv` class. The `BaseEnv` class leverages the `nocturne` simulator to create a basic RL interface, based on the provided traffic scenario(s). 

---
> 📝 The `env_config.yaml` file defines our environment settings, such as the action space, observation space and traffic scenarios to use.
---

Check out `configs/env_config` for all the details!

In [2]:
import yaml
from nocturne.envs.base_env import BaseEnv

# Load environment settings
with open(f"../configs/env_config.yaml", "r") as stream:
    env_config = yaml.safe_load(stream)

# Initialize environment
env = BaseEnv(config=env_config)

In [3]:
print(f'controlling agents # {[agent.id for agent in env.controlled_vehicles]}')

controlling agents # [32, 3]


#### **Data**

- Within `env_config.yaml`, we specify the path to the folder containing the traffic scenarios to use as follows:

```yaml
# Path to folder with traffic scene(s) from which to create an environment
data_path: ../data
```

- [Here](https://github.com/facebookresearch/nocturne/tree/main#downloading-the-dataset) are the instructions to access the complete dataset of traffic scenes. 

- The data folder also has a file named `valid_files.json`. This file lists the names of all the valid traffic scenarios along with the ids of the vehicles that are not valid. These vehicles are excluded from our experiment.

For simplicity, we currently added a single traffic scenario that includes two vehicles in our data folder. Both vehicles can be used, so our `valid_files.json` looks like this:

```yaml
{
    "example_scenario.json": []
}
```

### Interacting with the environment

The classic agent-environment loop of reinforcement learning is implemented as follows:

In [4]:
# Reset
obs_dict = env.reset()

# Get info
agent_ids = [agent_id for agent_id in obs_dict.keys()]
dead_agent_ids = []
num_agents = len(agent_ids)
rewards = {agent_id: 0 for agent_id in agent_ids}

for step in range(1000):

    # Sample actions
    action_dict = {
        agent_id: env.action_space.sample() 
        for agent_id in agent_ids
        if agent_id not in dead_agent_ids
    }
    
    # Step in env
    obs_dict, rew_dict, done_dict, info_dict = env.step(action_dict)

    for agent_id in action_dict.keys():
        rewards[agent_id] += rew_dict[agent_id]

    # Update dead agents
    for agent_id, is_done in done_dict.items():
        if is_done and agent_id not in dead_agent_ids:
            dead_agent_ids.append(agent_id)

    # Reset if all agents are done
    if done_dict["__all__"]:
        print(f'Done after {env.step_num} steps -- total return in episode: {rewards}')
        obs_dict = env.reset()
        dead_agent_ids = []
        rewards = {agent_id: 0 for agent_id in agent_ids}

# Close environment
env.close()

Done after 80 steps -- total return in episode: {3: 1.7939045316631903, 32: 1.76165686989652}
Done after 80 steps -- total return in episode: {3: 2.155318604202339, 32: 1.481429297893312}
Done after 80 steps -- total return in episode: {3: 9.465985459353316, 32: 2.2998212474249136}
Done after 80 steps -- total return in episode: {3: 1.7018343298612015, 32: 1.3247767658493788}
Done after 80 steps -- total return in episode: {3: 1.8384227205755483, 32: 10.332866871900634}
Done after 80 steps -- total return in episode: {3: 1.1086751511448438, 32: 2.2523170773066994}
Done after 68 steps -- total return in episode: {3: 9.61291282706631, 32: 1.7437541483099983}
Done after 80 steps -- total return in episode: {3: 1.3500529425191474, 32: 1.4489859636190936}
Done after 80 steps -- total return in episode: {3: 0.2037829695907602, 32: 1.79063201755183}
Done after 80 steps -- total return in episode: {3: 0.5679890269139611, 32: 1.1160696685449862}
Done after 80 steps -- total return in episode: {

### Accessing information about the environment

In [5]:
# The observation space 
env.observation_space


Box(-inf, inf, (10,), float32)

In [6]:
# The size of the joint action space 
env.action_space


Discrete(9)

In [7]:
# Which agents are controlled?
env.controlled_vehicles

[<nocturne_cpp.Vehicle at 0x135413430>, <nocturne_cpp.Vehicle at 0x135413eb0>]

### 
